#Introduction

This notebook has all the code you need to create your own chatbot with custom knowledge base using GPT-3. 

Follow the instructions for each steps and then run the code sample. In order to run the code, you need to press "play" button near each code sample.

# New Section

#Download the data for your custom knowledge base
For the demonstration purposes we are going to use ----- as our knowledge base. You can download them to your local folder from the github repository by running the code below.
Alternatively, you can put your own custom data into the local folder. 

In [1]:
! git clone https://github.com/steelblu/DFOCUS_QA.git

Cloning into 'DFOCUS_QA'...


# Install the dependicies
Run the code below to install the depencies we need for our functions

In [7]:
%pip install gpt-index
%pip install langchain
%pip install transformers

  Using cached gpt_index-0.4.23-py3-none-any.whl (231 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Collecting transformers
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)



ERROR: Could not install packages due to an OSError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'c:\\Python311\\Scripts\\transformers-cli.exe' -> 'c:\\Python311\\Scripts\\transformers-cli.exe.deleteme'


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Define the functions
The following code defines the functions we need to construct the index and query it

In [8]:
from gpt_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 300
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600 

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

    index.save_to_disk('index.json')

    return index

def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True: 
        query = input("What do you want to ask? ")
        response = index.query(query, response_mode="compact")
        display(Markdown(f"Response: <b>{response.response}</b>"))
  

# Set OpenAI API Key
You need an OPENAI API key to be able to run this code.

If you don't have one yet, get it by [signing up](https://platform.openai.com/overview). Then click your account icon on the top right of the screen and select "View API Keys". Create an API key.

Then run the code below and paste your API key into the text input.

In [11]:
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

#Construct an index
Now we are ready to construct the index. This will take every file in the folder 'data', split it into chunks, and embed it with OpenAI's embeddings API.

**Notice:** running this code will cost you credits on your OpenAPI account ($0.02 for every 1,000 tokens). If you've just set up your account, the free credits that you have should be more than enough for this experiment.

In [22]:
construct_index("DFOCUS_QA/data")

#Ask questions
It's time to have fun and test our AI. Run the function that queries GPT and type your question into the input. 

If you've used the provided example data for your custom knowledge base, here are a few questions that you can ask:
1. seedCloud는?
2. 디포커스는?


In [23]:
ask_ai()

What do you want to ask? seedCloud가 뭐야


Response: <b>
SeedCloud는 포털 솔루션으로, 다양한 오토스케일러 그룹을 관리하고 CPU, Memory 기반의 자원을 증가시켜주는 Scale-Up/Out 기능과 사용자 정의 기능, 커스텀 작업 처리 옵션 기능, IP Pool을 기반으로 한 자동화된 IP 충돌 방지 기능을 제공하는 플</b>

What do you want to ask? 대한민국 대통령은?


Response: <b>
문재인입니다.</b>

What do you want to ask? 지금 몇년도야


...

What do you want to ask? 전화번호


Response: <b>
010-7211-0616</b>

What do you want to ask? 전화


Response: <b>문의는 어디로 하면 됩니까?

전화 문의는 010-7211-0616로 하시면 됩니다.</b>

KeyboardInterrupt: ignored